In [6]:
import numpy as np
import time
import pandas as pd
import random
from itertools import product

import warnings

warnings.filterwarnings('ignore')

In [13]:
def gerar_todas_matrizes_binarias(linhas, colunas):
    """
    Gera todas as possibilidades de matrizes binárias de tamanho (linhas, colunas).
    """
    # O número total de elementos na matriz
    tamanho_total = linhas * colunas
    
    # Gera todas as sequências binárias possíveis do tamanho total
    # repeat=tamanho_total garante o comprimento correto para cada combinação
    for valores in product([0, 1], repeat=tamanho_total):
        # Converte a tupla de valores em um array numpy e remodela para a matriz
        matriz = np.array(valores).reshape((linhas, colunas))
        yield matriz

In [ ]:
def termicas_guloso(df_terms,demanda,size):

    total_tempo = len(demanda)
    colunas = list(df_terms.termica.unique()) ### Térmicas
    colunas_demanda = list(map(lambda item: 'demanda_' + item, colunas)) ### Demanda utilizada de cada térmica em cada tempo
    colunas_toff = list(map(lambda item: 'tempo_desligada_' + item, colunas)) ### Tempo que cada térmica está desligada
    colunas.extend(colunas_demanda)
    colunas.extend(colunas_toff)
    colunas.extend(['tempo','demanda','custo'])
    df_tempo = pd.DataFrame(columns=colunas,index=range(size)) ### Tabela final com os resultados de cada período de tempo
    df_terms_ordenado = df_terms.sort_values('custo') ### Ordenamento por melhor custo

    ### Iteração para cada período de tempo
    for tempo in range(total_tempo):

        demanda_temp = 0
        custo = 0
        demanda_atual = demanda[tempo]

        if(tempo == 0): ### Condição especial quando o tempo é igual a zero e nenhuma térmica foi acionada ou deve permanecer desligada
            for term in range(len(df_terms_ordenado)): ### Iteração para cada térmica em cada período de tempo
                limite_superior = df_terms_ordenado.limite_superior.iloc[term]
                limite_inferior = df_terms_ordenado.limite_inferior.iloc[term]
                custo_term = df_terms_ordenado.custo.iloc[term]
                termica = df_terms_ordenado.termica.iloc[term]
                t_off = df_terms_ordenado.t_off.iloc[term]
                df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = t_off
            
                if(((limite_superior+demanda_temp)>demanda_atual) and ## Demanda atual é inferior a soma do limite superior e a demanda já preenchida
                (demanda_temp != demanda_atual) and 
                ((demanda_atual-demanda_temp)>=limite_inferior)):
                    demanda_utilizada = demanda_atual-demanda_temp
                    custo = custo + (custo_term*demanda_utilizada)
                    demanda_temp = demanda_temp+demanda_utilizada
                    df_tempo.iloc[tempo][termica] = 1
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0 
                    print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp,
                          "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                    
                elif((demanda_temp<demanda_atual) and ## Demanda preenchida é maior que limite inferior
                ((demanda_atual-demanda_temp)>=limite_inferior)):
                    demanda_utilizada = limite_superior
                    custo = custo+(custo_term*demanda_utilizada)
                    demanda_temp = demanda_temp+demanda_utilizada
                    df_tempo.iloc[tempo][termica] = 1
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0 
                    print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                          "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                
                else: ### Térmica não foi acionada
                    df_tempo.iloc[tempo][termica] = 0
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = 0
            
            if(demanda_atual-demanda_temp!=0): ### Não foi possível encontrar uma combinação para atender a demanda
                return print("PROBLEMA IMPOSSÍVEL DE RESOLVER!!")
        
        else: ### Demais períodos de tempo
            for term in range(len(df_terms_ordenado)): ### Iteração para cada térmica em cada período de tempo
                
                ##Definição da térmica de menor custo para preencher a demanda quando houver obrigatoriedade de acionar uma outra térmica mais cara
                if term == 0: 
                    melhor_term = df_terms_ordenado.termica.iloc[term]
                    melhor_custo = df_terms_ordenado.custo.iloc[term]
                
                limite_superior = df_terms_ordenado.limite_superior.iloc[term]
                limite_inferior = df_terms_ordenado.limite_inferior.iloc[term]
                custo_term = df_terms_ordenado.custo.iloc[term]
                termica = df_terms_ordenado.termica.iloc[term]
                t_on = df_terms_ordenado.t_on.iloc[term]
                t_off = df_terms_ordenado.t_off.iloc[term]
                tempo_ligada = df_tempo.iloc[tempo-1][termica]
                tempo_desligada = df_tempo.iloc[tempo-1][f'tempo_desligada_{termica}']
    
                if(tempo_ligada>0) and (tempo_ligada<t_on): ### Térmica ligada no último período e ainda não atingiu o T_On
                    if((demanda_atual-demanda_temp)>limite_superior): ### Demanda restante é maior que o limite superior da térmica
                        demanda_utilizada = limite_superior
                        custo = custo + (custo_term*demanda_utilizada)
                        demanda_temp = demanda_temp+demanda_utilizada
                        df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                        df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                        df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                        print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                              "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                    elif((demanda_atual-demanda_temp)<limite_inferior): ### Demanda restante é menor que o limite inferior da térmica
                        demanda_excedida = limite_inferior - (demanda_atual-demanda_temp)
                        demanda_utilizada = limite_inferior
                        demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                        custo = custo + (custo_term*demanda_utilizada) - (melhor_custo*demanda_excedida)
                        df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                        df_tempo.iloc[tempo][f'demanda_{melhor_term}'] = df_tempo.iloc[tempo][f'demanda_{melhor_term}']-demanda_excedida
                        df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                        df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                        df_tempo.iloc[tempo][f'tempo_desligada_{melhor_term}'] = 0
                        print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                          "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                    else: ### Demanda restante está entre o limite inferior e superior da térmica
                        demanda_utilizada = demanda_atual-demanda_temp
                        custo = custo + (custo_term*demanda_utilizada)
                        demanda_temp = demanda_temp+demanda_utilizada
                        df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                        df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                        df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                        print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                              "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])

                ### Térmica estava desligada e resta demanda
                elif(((limite_superior+demanda_temp)>demanda_atual) and  ### Demanda restante mais limite superior é maior que demanda atual
                (demanda_temp != demanda_atual) and 
                ((demanda_atual-demanda_temp)>=limite_inferior) and
                ((tempo_desligada>=t_off) or (tempo_desligada==0))):
                    demanda_utilizada = demanda_atual-demanda_temp
                    custo = custo + (custo_term*demanda_utilizada)
                    demanda_temp = demanda_temp+demanda_utilizada
                    df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                    print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                          "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                    
                elif((demanda_temp<demanda_atual) and ### Demanda restante é maior que o limite inferior
                ((demanda_atual-demanda_temp)>=limite_inferior) and
                ((tempo_desligada>=t_off) or (tempo_desligada==0))):
                    demanda_utilizada = limite_superior
                    custo = custo+(custo_term*demanda_utilizada)
                    demanda_temp = demanda_temp+demanda_utilizada
                    df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                    print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp,
                          "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])

                elif((demanda_temp<demanda_atual) and ### Demanda restante é menor que o limite inferior
                     (demanda_atual-demanda_temp<limite_inferior) and
                     ((tempo_desligada>=t_off) or (tempo_desligada==0))):
                    demanda_excedida = limite_inferior - (demanda_atual-demanda_temp)
                    demanda_utilizada = limite_inferior
                    demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                    custo = custo + (custo_term*demanda_utilizada) - (melhor_custo*demanda_excedida)
                    df_tempo.iloc[tempo][termica] = df_tempo.iloc[tempo-1][termica]+1
                    df_tempo.iloc[tempo][f'demanda_{melhor_term}'] = df_tempo.iloc[tempo][f'demanda_{melhor_term}']-demanda_excedida
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = 0
                    df_tempo.iloc[tempo][f'tempo_desligada_{melhor_term}'] = 0
                    print("Térmica acionada: ",termica,"Demanda atual: ",demanda_atual,"Demanda temp: ",demanda_temp, 
                      "Tempo: ",tempo,"Custo: ",custo,"Tempo acionada: ",df_tempo.iloc[tempo][termica])
                
                elif(tempo_desligada<=t_off) and (demanda_atual!=demanda_temp): ### Térmica não foi acionada por conta de t_off
                    print("Térmica: ",termica, " não pôde ser acionada no tempo: ",tempo,
                          " porque estava desligada a : ",tempo_desligada," períodos e deve ficar desligada por : ",t_off," períodos")
                    df_tempo.iloc[tempo][termica] = 0
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = 0
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = tempo_desligada + 1
                
                else: ### Térmica não foi acionada neste período
                    df_tempo.iloc[tempo][termica] = 0
                    df_tempo.iloc[tempo][f'demanda_{termica}'] = 0
                    df_tempo.iloc[tempo][f'tempo_desligada_{termica}'] = tempo_desligada + 1
            
            if(demanda_atual-demanda_temp!=0): ### Não foi possível encontrar uma combinação para atender a demanda
                return print("PROBLEMA IMPOSSÍVEL DE RESOLVER!!!!!!")
            
        df_tempo.iloc[tempo]['tempo'] = tempo
        df_tempo.iloc[tempo]['demanda'] = demanda[tempo]
        df_tempo.iloc[tempo]['custo'] = custo

    #df_tempo['soma_demanda'] = df_tempo.demanda_term1+df_tempo.demanda_term2+df_tempo.demanda_term3+df_tempo.demanda_term4+df_tempo.demanda_term5
    #df_tempo['dif_demanda'] = df_tempo.demanda-df_tempo.soma_demanda
    
    return df_tempo

In [ ]:
def busca_local(df_resultado,df_regras): ### Busca local com parada em primeira melhoria

    termicas = list(df_regras.termica.unique())
    df_regras_ordenado = df_regras.sort_values('custo')
    termicas_ordenadas_melhor_custo = df_regras_ordenado.termica.unique()
    #df_vizinhos_final = df_resultado
    i=0
    
    for tempo in range(len(df_resultado)):
        custo_tempo_atual = df_resultado.iloc[tempo]['custo']
        demanda = df_resultado.iloc[tempo]['demanda']
        
        for term in termicas:
            
            df_novo_vizinho = df_resultado.copy()
            
            if df_novo_vizinho.iloc[tempo][term]==1:
                df_novo_vizinho.iloc[tempo][term] = 0
                df_novo_vizinho.iloc[tempo][f'demanda_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_ligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_desligada_{term}'] + 1
            else:
                df_novo_vizinho.iloc[tempo][term]==1
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] + 1

            if(solucao_viavel(df_novo_vizinho,df_regras)):

                print("Solução viável encontrada no tempo: ",tempo," na iteração :",i)
                demanda_utilizada = 0
                demanda_temp = 0
                custo = 0
                df_temporario = pd.DataFrame(columns=['termica','demanda','limite_inferior','custo'])
                
                for termica in termicas_ordenadas_melhor_custo:

                    if(df_novo_vizinho.iloc[tempo][termica]==1):
                        
                        limite_superior = df_regras.loc[(df_regras.termica==termica)].limite_superior.iloc[0]
                        limite_inferior = df_regras.loc[(df_regras.termica==termica)].limite_inferior.iloc[0]
                        custo_term = df_regras.loc[(df_regras.termica==termica)].custo.iloc[0]
    
                        if((demanda_temp+limite_superior)<=demanda):
                            demanda_utilizada = limite_superior
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        elif((demanda_temp+limite_superior>demanda) and (demanda_temp+limite_inferior<=demanda)):
                            demanda_utilizada = demanda - demanda_temp
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        else:
                            demanda_excedida = limite_inferior - (demanda-demanda_temp)
                            demanda_utilizada = limite_inferior
                            demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
              
                            df_temporario.custo = df_temporario.custo.astype('int64')
                            pior_termica = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['termica'].iloc[0]
                            limite_inferior_pior_termica = df_temporario.loc[df_temporario.custo==\
                                                           df_temporario.custo.max()]['limite_inferior'].iloc[0]
                            
                            if(df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}']>=(demanda_excedida+limite_inferior_pior_termica)):
                                pior_custo = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida
                            else:
                                pior_termica = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['termica'].iloc[0]
                                limite_inferior_pior_termica = df_temporario.loc[(df_temporario.custo==\
                                                               df_temporario.custo.nlargest().iloc[1])]['limite_inferior'].iloc[0]
                                pior_custo = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida

                            custo = custo + (custo_term*demanda_utilizada) - (pior_custo*demanda_excedida)
                            df_novo_vizinho.iloc[tempo]['custo'] = custo

                if custo < custo_tempo_atual:
                    print("Foi encontrado uma solução melhor!")
                    print("Solução melhor encontrada no tempo: ",tempo," custo atual: ",custo," custo anterior:",custo_tempo_atual)
                    return df_novo_vizinho

            i=i+1

    return print("Não foi encontrado uma solução melhor!")

In [8]:
def perturbacao(df, df_regras, n_mudancas):
    #Perturba a solução, invertendo a escolha de itens aleatórios
    df_solucao_perturbada = df.copy()
    n_periodos = len(df)
    tempos_para_mudar = random.sample(range(n_periodos), n_mudancas)
    termicas = list(df_regras.termica.unique())
    term_para_mudar = random.sample(termicas,2)
    
    for tempo in tempos_para_mudar:
        for termica in term_para_mudar:
            df_solucao_perturbada.iloc[tempo][termica] = 1 - df_solucao_perturbada.iloc[tempo][termica]
    
    return df_solucao_perturbada

In [ ]:
def perturbacao(df, df_regras, n_mudancas):
    #Perturba a solução, invertendo a escolha de itens aleatórios
    df_solucao_perturbada = df.copy()
    n_periodos = len(df)
    tempos_para_mudar = random.sample(range(n_periodos), n_mudancas)
    termicas = list(df_regras.termica.unique())
    term_para_mudar = random.sample(termicas,2)
    df_regras_ordenado = df_regras.sort_values('custo')
    termicas_ordenadas_melhor_custo = df_regras_ordenado.termica.unique()
    
    for tempo in tempos_para_mudar:
        
        custo_tempo_atual = df_solucao_perturbada.iloc[tempo]['custo']
        demanda = df_solucao_perturbada.iloc[tempo]['demanda']
        
        for termica in term_para_mudar:
            df_solucao_perturbada.iloc[tempo][termica] = 1 - df_solucao_perturbada.iloc[tempo][termica]
        
        if solucao_viavel(df_solucao_perturbada,df_regras):

            #print("Solução viável encontrada na perturbação no tempo: ",tempo)
            demanda_utilizada = 0
            demanda_temp = 0
            custo = 0
            df_temporario = pd.DataFrame(columns=['termica','demanda','limite_inferior','custo'])
            
            for termica in termicas_ordenadas_melhor_custo:
                
                if(df_solucao_perturbada.iloc[tempo][termica]==1): 

                    limite_superior = df_regras.loc[(df_regras.termica==termica)].limite_superior.iloc[0]
                    limite_inferior = df_regras.loc[(df_regras.termica==termica)].limite_inferior.iloc[0]
                    custo_term = df_regras.loc[(df_regras.termica==termica)].custo.iloc[0]

                    if((demanda_temp+limite_superior)<=demanda):
                            demanda_utilizada = limite_superior
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_solucao_perturbada.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_solucao_perturbada.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_solucao_perturbada.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                    elif((demanda_temp+limite_superior>demanda) and (demanda_temp+limite_inferior<=demanda)):
                        demanda_utilizada = demanda - demanda_temp
                        demanda_temp = demanda_temp + demanda_utilizada
                        custo = custo + (demanda_utilizada*custo_term)
                        df_solucao_perturbada.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                        df_solucao_perturbada.iloc[tempo]['custo'] = custo
                        df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                       'demanda':df_solucao_perturbada.iloc[tempo][f'demanda_{termica}'],
                                                                       'custo':custo_term},index=[1])])

                    else:
                        demanda_excedida = limite_inferior - (demanda-demanda_temp)
                        demanda_utilizada = limite_inferior
                        demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                        df_solucao_perturbada.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
          
                        df_temporario.custo = df_temporario.custo.astype('int64')
                        pior_termica = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['termica'].iloc[0]
                        limite_inferior_pior_termica = df_temporario.loc[df_temporario.custo==\
                                                       df_temporario.custo.max()]['limite_inferior'].iloc[0]
                        
                        if(df_solucao_perturbada.iloc[tempo][f'demanda_{pior_termica}']>=(demanda_excedida+limite_inferior_pior_termica)):
                            pior_custo = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['custo'].iloc[0]
                            df_solucao_perturbada.iloc[tempo][f'demanda_{pior_termica}'] = \
                            df_solucao_perturbada.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida
                        else:
                            pior_termica = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['termica'].iloc[0]
                            limite_inferior_pior_termica = df_temporario.loc[(df_temporario.custo==\
                                                           df_temporario.custo.nlargest().iloc[1])]['limite_inferior'].iloc[0]
                            pior_custo = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['custo'].iloc[0]
                            df_solucao_perturbada.iloc[tempo][f'demanda_{pior_termica}'] = \
                            df_solucao_perturbada.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida

                        custo = custo + (custo_term*demanda_utilizada) - (pior_custo*demanda_excedida)
                        df_solucao_perturbada.iloc[tempo]['custo'] = custo
                    
            if custo < custo_tempo_atual:
                print("Foi encontrado uma solução melhor na perturbação!")
                print("Solução melhor encontrada no tempo: ",tempo," custo atual: ",custo," custo anterior:",custo_tempo_atual)
                custo_tempo_atual = custo
                return df_solucao_perturbada
            
    
    return df_solucao_perturbada

In [ ]:
def busca_local(df_resultado,df_regras,max): ### Busca local com parada em máximo de iterações

    termicas = list(df_regras.termica.unique())
    df_regras_ordenado = df_regras.sort_values('custo')
    termicas_ordenadas_melhor_custo = df_regras_ordenado.termica.unique()
    df_melhor_vizinho = df_resultado.copy()
    i=0
    max_it = 0
    lista_resultado = list(df_resultado.index)
    random.shuffle(lista_resultado)
    #lista_resultado = [17] + [47] + lista_resultado

    for tempo in lista_resultado:
            
        custo_tempo_atual = df_resultado.iloc[tempo]['custo']
        demanda = df_resultado.iloc[tempo]['demanda']
        
        for term in termicas:
            
            df_novo_vizinho = df_melhor_vizinho.copy()
            
            if df_novo_vizinho.iloc[tempo][term]==1:
                df_novo_vizinho.iloc[tempo][term] = 0
                df_novo_vizinho.iloc[tempo][f'demanda_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_ligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_desligada_{term}'] + 1
            else:
                df_novo_vizinho.iloc[tempo][term]=1
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] + 1

            if(solucao_viavel(df_novo_vizinho,df_regras)):

                print("Solução viável encontrada no tempo: ",tempo," na iteração :",i)
                demanda_utilizada = 0
                demanda_temp = 0
                custo = 0
                df_temporario = pd.DataFrame(columns=['termica','demanda','limite_inferior','custo'])
                
                for termica in termicas_ordenadas_melhor_custo:

                    if(df_novo_vizinho.iloc[tempo][termica]==1):
                        
                        limite_superior = df_regras.loc[(df_regras.termica==termica)].limite_superior.iloc[0]
                        limite_inferior = df_regras.loc[(df_regras.termica==termica)].limite_inferior.iloc[0]
                        custo_term = df_regras.loc[(df_regras.termica==termica)].custo.iloc[0]
    
                        if((demanda_temp+limite_superior)<=demanda):
                            demanda_utilizada = limite_superior
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        elif((demanda_temp+limite_superior>demanda) and (demanda_temp+limite_inferior<=demanda)):
                            demanda_utilizada = demanda - demanda_temp
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        else:
                            demanda_excedida = limite_inferior - (demanda-demanda_temp)
                            demanda_utilizada = limite_inferior
                            demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
              
                            df_temporario.custo = df_temporario.custo.astype('int64')
                            pior_termica = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['termica'].iloc[0]
                            limite_inferior_pior_termica = df_temporario.loc[df_temporario.custo==\
                                                           df_temporario.custo.max()]['limite_inferior'].iloc[0]
                            
                            if(df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}']>=(demanda_excedida+limite_inferior_pior_termica)):
                                pior_custo = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida
                            else:
                                pior_termica = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['termica'].iloc[0]
                                limite_inferior_pior_termica = df_temporario.loc[(df_temporario.custo==\
                                                               df_temporario.custo.nlargest().iloc[1])]['limite_inferior'].iloc[0]
                                pior_custo = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida

                            custo = custo + (custo_term*demanda_utilizada) - (pior_custo*demanda_excedida)
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                    
                    i=i+1
                    max_it=max_it+1
                    #print("Max It = ",max_it)
                    
                    if(max_it>=max):
                        if(df_melhor_vizinho.custo.sum()<df_resultado.custo.sum()):
                            return df_melhor_vizinho
                        else:
                            print("Não foi encontrado uma solução melhor!")
                            return df_resultado
                
                if custo < custo_tempo_atual:
                    print("Foi encontrado uma solução melhor!")
                    print("Solução melhor encontrada no tempo: ",tempo," custo atual: ",custo," custo anterior:",custo_tempo_atual)
                    custo_tempo_atual = custo
                    max_it = 0
                    df_melhor_vizinho = df_novo_vizinho.copy()
                    return df_novo_vizinho
                    
            else:    
                i=i+1
                max_it = max_it+1
            
            if(max_it>=max):
                    if(df_melhor_vizinho.custo.sum()<df_resultado.custo.sum()):
                        return df_melhor_vizinho
                    else:
                        print("Não foi encontrado uma solução melhor!")
                        return df_resultado

            #print("Max It = ",max_it)
                

    if(df_melhor_vizinho.custo.sum()<df_resultado.custo.sum()):
        return df_melhor_vizinho
    else:
        print("Não foi encontrado uma solução melhor!")
        return df_resultado

In [ ]:
def busca_local(df_resultado,df_regras,max): ### Busca local com parada em máximo de iterações

    termicas = list(df_regras.termica.unique())
    df_regras_ordenado = df_regras.sort_values('custo')
    termicas_ordenadas_melhor_custo = df_regras_ordenado.termica.unique()
    df_melhor_vizinho = df_resultado.copy()
    i=0
    max_it = 0
    lista_resultado = list(df_resultado.index)
    random.shuffle(termicas)
    custo_total = df_resultado.custo.sum()
    #lista_resultado = [17] + [47] + lista_resultado

    for term in termicas:
        df_novo_vizinho = df_melhor_vizinho.copy()
        for tempo in lista_resultado:
                        
            demanda = df_resultado.iloc[tempo]['demanda']
            #df_novo_vizinho = df_melhor_vizinho.copy()
            
            if df_novo_vizinho.iloc[tempo][term]==1 and df_novo_vizinho.iloc[tempo2][term]==0:
                df_novo_vizinho.iloc[tempo][term] = 0
                df_novo_vizinho.iloc[tempo][f'demanda_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_ligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_desligada_{term}'] + 1
            else:
                df_novo_vizinho.iloc[tempo][term]=1
                df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = 0
                df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}'] + 1

            for tempos in range(len(df_resultado)):
                if tempos>0:
                    if df_novo_vizinho.iloc[tempos][term] == 0:
                        df_novo_vizinho.iloc[tempos][f'tempo_ligada_{term}'] = 0
                        df_novo_vizinho.iloc[tempos][f'tempo_desligada_{term}'] = df_novo_vizinho.iloc[tempos-1][f'tempo_desligada_{term}']+1
                    else:
                        df_novo_vizinho.iloc[tempo][f'tempo_ligada_{term}'] = df_novo_vizinho.iloc[tempo-1][f'tempo_ligada_{term}']+1
                        df_novo_vizinho.iloc[tempo][f'tempo_desligada_{term}'] = 0

            if(solucao_viavel(df_novo_vizinho,df_regras)):

                print("Solução viável encontrada no tempo: ",tempo,",mudança na térmica: ",term," na iteração :",i)
                demanda_utilizada = 0
                demanda_temp = 0
                custo = 0
                df_temporario = pd.DataFrame(columns=['termica','demanda','limite_inferior','custo'])
                
                for termica in termicas_ordenadas_melhor_custo:

                    if(df_novo_vizinho.iloc[tempo][termica]==1):
                        
                        limite_superior = df_regras.loc[(df_regras.termica==termica)].limite_superior.iloc[0]
                        limite_inferior = df_regras.loc[(df_regras.termica==termica)].limite_inferior.iloc[0]
                        custo_term = df_regras.loc[(df_regras.termica==termica)].custo.iloc[0]
    
                        if((demanda_temp+limite_superior)<=demanda):
                            demanda_utilizada = limite_superior
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        elif((demanda_temp+limite_superior>demanda) and (demanda_temp+limite_inferior<=demanda)):
                            demanda_utilizada = demanda - demanda_temp
                            demanda_temp = demanda_temp + demanda_utilizada
                            custo = custo + (demanda_utilizada*custo_term)
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                            df_temporario = pd.concat([df_temporario,pd.DataFrame({'termica':termica,'limite_inferior':limite_inferior,
                                                                           'demanda':df_novo_vizinho.iloc[tempo][f'demanda_{termica}'],
                                                                           'custo':custo_term},index=[1])])

                        else:
                            demanda_excedida = limite_inferior - (demanda-demanda_temp)
                            demanda_utilizada = limite_inferior
                            demanda_temp = demanda_temp-demanda_excedida+demanda_utilizada
                            df_novo_vizinho.iloc[tempo][f'demanda_{termica}'] = demanda_utilizada
              
                            df_temporario.custo = df_temporario.custo.astype('int64')
                            pior_termica = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['termica'].iloc[0]
                            limite_inferior_pior_termica = df_temporario.loc[df_temporario.custo==\
                                                           df_temporario.custo.max()]['limite_inferior'].iloc[0]
                            
                            if(df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}']>=(demanda_excedida+limite_inferior_pior_termica)):
                                pior_custo = df_temporario.loc[df_temporario.custo==df_temporario.custo.max()]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida
                            else:
                                pior_termica = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['termica'].iloc[0]
                                limite_inferior_pior_termica = df_temporario.loc[(df_temporario.custo==\
                                                               df_temporario.custo.nlargest().iloc[1])]['limite_inferior'].iloc[0]
                                pior_custo = df_temporario.loc[(df_temporario.custo==df_temporario.custo.nlargest().iloc[1])]['custo'].iloc[0]
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] = \
                                df_novo_vizinho.iloc[tempo][f'demanda_{pior_termica}'] - demanda_excedida

                            custo = custo + (custo_term*demanda_utilizada) - (pior_custo*demanda_excedida)
                            df_novo_vizinho.iloc[tempo]['custo'] = custo
                    
                    i=i+1
                    max_it=max_it+1
                    #print("Max It = ",max_it)
                    
                    if(max_it>=max):
                        if(df_melhor_vizinho.custo.sum()<custo_total):
                            return df_melhor_vizinho
                        else:
                            print("Não foi encontrado uma solução melhor!")
                            return df_resultado
                
                if df_novo_vizinho.custo.sum() < custo_total:
                    print("Foi encontrado uma solução melhor!")
                    print("Solução melhor encontrada no tempo: ",tempo,"mudança na térmica: ",term," custo atual: ",df_novo_vizinho.custo.sum(),
                          " custo anterior:",custo_total)
                    custo_total = df_melhor_vizinho.custo.sum()
                    max_it = 0
                    df_melhor_vizinho = df_novo_vizinho.copy()
                    return df_melhor_vizinho
                    
            else:    
                i=i+1
                max_it = max_it+1
            
            if(max_it>=max):
                    if(df_melhor_vizinho.custo.sum()<df_resultado.custo.sum()):
                        return df_melhor_vizinho
                    else:
                        print("Não foi encontrado uma solução melhor!!")
                        return df_resultado

            #print("Max It = ",max_it)
                

    if(df_melhor_vizinho.custo.sum()<df_resultado.custo.sum()):
        return df_melhor_vizinho
    else:
        print("Não foi encontrado uma solução melhor!!!")
        return df_resultado

In [3]:
df_term_david = pd.DataFrame({'termica':['g1','g2','g3'],'limite_inferior':[10,10,10],'limite_superior':[100,100,100],
                             't_on':[2,5,1],'t_off':[1,1,1],'custo':[10,20,24]})

In [4]:
df_term_david

,termica,limite_inferior,limite_superior,t_on,t_off,custo
0,g1,10,100,2,1,10
1,g2,10,100,5,1,20
2,g3,10,100,1,1,24


In [5]:
demanda_david = [90,90,110,90,90,90,90,90]

In [7]:
df_term_david.to_csv("df_term_david.csv",index=False)
np.savetxt("demanda_david.csv", demanda_david, delimiter=",")